In [146]:
import json
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.stats import linregress

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


## Extract data

In [2]:
files = glob.glob(r'C:\Users\trist\OneDrive\Documents\game_data\valorant_raw/*.json')
len(files)

2403

In [3]:
with open(files[0], 'r') as f:
    json_data = json.load(f)
json_data = json.loads(json_data)
json_data['data'].keys()

dict_keys(['attributes', 'metadata', 'segments', 'expiryDate'])

In [4]:
files[0]

'C:\\Users\\trist\\OneDrive\\Documents\\game_data\\valorant_raw\\0001be30-1ef6-40d8-8f92-13ffbd093cf1.json'

In [5]:
json_data['data']['attributes']

{'id': '0001be30-1ef6-40d8-8f92-13ffbd093cf1'}

In [6]:
json_data['data']['metadata']

{'modeKey': 'bomb',
 'modeName': 'Normal',
 'modeImageUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/icons/modes/normal.png',
 'modeMaxRounds': 25,
 'duration': 2664924,
 'dateStarted': '2022-07-08T06:53:09.588+00:00',
 'rounds': 24,
 'isRanked': False,
 'queueId': 'unrated',
 'map': '2fb9a4fd-47b8-4e7d-a969-74b4046ebd53',
 'mapName': 'Breeze',
 'mapImageUrl': 'https://titles.trackercdn.com/valorant-api/maps/2fb9a4fd-47b8-4e7d-a969-74b4046ebd53/splash.png'}

In [7]:
game_metadata = json_data['data']['metadata']

In [8]:
json_data['data']['expiryDate']

'2022-08-08T18:11:36.1374295+00:00'

In [9]:
all_segments_types = set()

for i in json_data['data']['segments']:
    all_segments_types.add(i['type'])

all_segments_types

{'player-round',
 'player-round-damage',
 'player-round-kills',
 'player-summary',
 'round-summary',
 'team-summary'}

In [10]:
player_rounds = [i for i in json_data['data']['segments'] if i['type'] == 'player-round']
player_rounds_damage = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-damage']
player_rounds_kills = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-kills']
player_summary = [i for i in json_data['data']['segments'] if i['type'] == 'player-summary']
round_summary = [i for i in json_data['data']['segments'] if i['type'] == 'round-summary']
team_summary = [i for i in json_data['data']['segments'] if i['type'] == 'team-summary']

## player_rounds_kills

In [11]:
player_rounds_kills[0].keys()

dict_keys(['type', 'attributes', 'metadata', 'expiryDate', 'stats'])

In [12]:



player_rounds_kills[0]['attributes']

{'round': 1,
 'platformSlug': 'riot',
 'platformUserIdentifier': 'rimo#SEXY',
 'opponentPlatformSlug': 'riot',
 'opponentPlatformUserIdentifier': 'Hennessyy#AKA47'}

In [13]:
player_rounds_kills[0]['metadata']

{'platformInfo': {'platformSlug': 'riot',
  'platformUserId': None,
  'platformUserHandle': 'rimo#SEXY',
  'platformUserIdentifier': 'rimo#SEXY',
  'avatarUrl': None,
  'additionalParameters': None},
 'opponentPlatformInfo': {'platformSlug': 'riot',
  'platformUserId': None,
  'platformUserHandle': 'Hennessyy#AKA47',
  'platformUserIdentifier': 'Hennessyy#AKA47',
  'avatarUrl': None,
  'additionalParameters': None},
 'opponentLocation': {'x': 4917, 'y': 3223},
 'playerLocations': [{'puuid': 'gaxPhPH2L0l07DbnPvzTzKcQzNi3qJfuNjigpnUV3UBkRNdDpGdhyhbU2KWVV1QbHIAXZrowoPOkcA',
   'viewRadians': 2.4492486,
   'location': {'x': 5927, 'y': 4912}},
  {'puuid': '-2GqRI2DorSneRUZgv2T47wOha66l9v3qUbG7Pxw1VibAdJkrHnLI6WgCZeAj3O8HUtAuAyuD4suLQ',
   'viewRadians': 5.6062036,
   'location': {'x': 4806, 'y': 5496}},
  {'puuid': 'LyIVoyMdyMEmaLGGwwI7eyO0vc6UIlwreIZF7OaidgY0SyLa4vF3Q6jGLlB_KBtCkhHfMtTNYaWi1A',
   'viewRadians': 5.0044217,
   'location': {'x': 3969, 'y': 6342}},
  {'puuid': 'q8cbMuUjAnP0Hj

In [14]:
player_rounds_kills[0]['stats']

{'damage': {'rank': None,
  'percentile': None,
  'displayName': None,
  'displayCategory': None,
  'category': None,
  'metadata': {},
  'value': 100,
  'displayValue': '100',
  'displayType': 'Number'}}

In [15]:
files = glob.glob(r'C:\Users\trist\OneDrive\Documents\game_data\valorant_raw/*.json')
print(len(files))
kill_records = list()

for file in files:
    with open(file, 'r') as f:
        json_data = json.load(f)
    json_data = json.loads(json_data)
    player_rounds_kills = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-kills']
    for kill in player_rounds_kills:        
        kill_records.append({'weaponName':kill['metadata']['weaponName'],
                            'round':kill['attributes']['round'],
                            'roundTime':kill['metadata']['roundTime']})
kill_df = pd.DataFrame.from_dict(kill_records)
kill_df.head()

2403


,weaponName,round,roundTime
0,Classic,1,20725
1,Ghost,1,37182
2,Classic,1,51285
3,Classic,1,23532
4,Classic,1,27015


In [16]:
kill_df['weaponName'].value_counts().index.tolist()

['Vandal',
 'Phantom',
 'Spectre',
 'Ghost',
 'Classic',
 'Operator',
 'Sheriff',
 'Guardian',
 'Marshal',
 'Odin',
 'Judge',
 'Bulldog',
 'Stinger',
 'Frenzy',
 'Ares',
 'Shorty',
 'Bucky']

## round_summary

In [17]:
round_summary[19]

{'type': 'round-summary',
 'attributes': {'round': 20},
 'metadata': {},
 'expiryDate': '0001-01-01T00:00:00+00:00',
 'stats': {'roundResult': {'rank': None,
   'percentile': None,
   'displayName': None,
   'displayCategory': None,
   'category': None,
   'metadata': {},
   'value': 'Detonate',
   'displayValue': 'Detonate',
   'displayType': 'String'},
  'winningTeam': {'rank': None,
   'percentile': None,
   'displayName': None,
   'displayCategory': None,
   'category': None,
   'metadata': {},
   'value': 'Blue',
   'displayValue': 'Blue',
   'displayType': 'String'}}}

## player_rounds

In [18]:
player_rounds[17]

{'type': 'player-round',
 'attributes': {'round': 2,
  'platformSlug': 'riot',
  'platformUserIdentifier': 'Corgi Tree#NA1'},
 'metadata': {'teamId': 'Red',
  'agentKey': '707eab51-4836-f488-046a-cda6bf494859',
  'platformInfo': {'platformSlug': 'riot',
   'platformUserId': None,
   'platformUserHandle': 'Corgi Tree#NA1',
   'platformUserIdentifier': 'Corgi Tree#NA1',
   'avatarUrl': None,
   'additionalParameters': None},
  'hasWon': False},
 'expiryDate': '0001-01-01T00:00:00+00:00',
 'stats': {'score': {'rank': None,
   'percentile': None,
   'displayName': 'Score',
   'displayCategory': None,
   'category': None,
   'metadata': {},
   'value': 300,
   'displayValue': '300',
   'displayType': 'Number'},
  'kills': {'rank': None,
   'percentile': None,
   'displayName': 'Kills',
   'displayCategory': None,
   'category': None,
   'metadata': {},
   'value': 1,
   'displayValue': '1',
   'displayType': 'Number'},
  'deaths': {'rank': None,
   'percentile': None,
   'displayName': 'Dea

## round_summary

In [19]:
len(round_summary)

24

In [20]:
# round_summary[22]


In [21]:
def get_winner(round_summary):
    return round_summary[-1]['stats']['winningTeam']['value']

get_winner(round_summary)

'Red'

## player_summary

In [22]:
len(player_summary)

10

In [23]:
player_summary[0].keys()

dict_keys(['type', 'attributes', 'metadata', 'expiryDate', 'stats'])

In [24]:
player_summary[0]['attributes']

{'platformSlug': 'riot', 'platformUserIdentifier': 'Hennessyy#AKA47'}

In [25]:
player_summary[0]['metadata']

{'partyId': '4155c5e2-bf8d-4736-acb1-8ec95e2576b1',
 'teamId': 'Blue',
 'agentKey': 'add6443a-41bd-e414-f6ad-e58d267f4e95',
 'agentName': 'Jett',
 'agentColor': '#98DCFF',
 'agentImageUrl': 'https://titles.trackercdn.com/valorant-api/agents/add6443a-41bd-e414-f6ad-e58d267f4e95/displayicon.png',
 'agentPortraitUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/db/agents/jett_portrait.png',
 'countryCode': None,
 'platformInfo': {'platformSlug': 'riot',
  'platformUserId': None,
  'platformUserHandle': 'Hennessyy#AKA47',
  'platformUserIdentifier': 'Hennessyy#AKA47',
  'avatarUrl': None,
  'additionalParameters': None}}

In [26]:
player_summary[0]['stats'].keys()

dict_keys(['rank', 'currRank', 'score', 'scorePerRound', 'killsPerRound', 'kills', 'deaths', 'assists', 'kdRatio', 'damage', 'damagePerRound', 'singleKills', 'doubleKills', 'tripleKills', 'quadraKills', 'pentaKills', 'multiKills', 'grenadeCasts', 'ability1Casts', 'ability2Casts', 'ultimateCasts', 'grenadeCastsPerRound', 'ability1CastsPerRound', 'ability2CastsPerRound', 'ultimateCastsPerRound', 'plants', 'defuses', 'firstKills', 'firstDeaths', 'esr', 'firstKillsPerRound', 'firstDeathsPerRound', 'econRating', 'hsAccuracy', 'kast'])

In [27]:
player_summary[0]['stats']['rank']

{'rank': None,
 'percentile': None,
 'displayName': 'Rating',
 'displayCategory': None,
 'category': 'mmr',
 'metadata': {'iconUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/icons/tiersv2/0.png'},
 'value': 'Unranked',
 'displayValue': 'Unranked',
 'displayType': 'String'}

In [28]:
player_summary[0]['stats']['score']

{'rank': None,
 'percentile': None,
 'displayName': 'Score',
 'displayCategory': None,
 'category': None,
 'metadata': {},
 'value': 7621,
 'displayValue': '7,621',
 'displayType': 'Number'}

In [29]:
player_summary[0]['stats']['scorePerRound']

{'rank': None,
 'percentile': None,
 'displayName': 'Score per Round',
 'displayCategory': None,
 'category': None,
 'metadata': {},
 'value': 317.5416666666667,
 'displayValue': '318',
 'displayType': 'Number'}

## Win prediction data set creation

In [30]:
all_agents_list = ['Astra',
         'Breach',
         'Brimstone',
         'Chamber',
         'Cypher',
         'Fade',
         'Jett',
         'KAY/O',
         'Killjoy',
         'Neon',
         'Omen',
         'Phoenix',
         'Raze',
         'Reyna',
         'Sage',
         'Skye',
         'Sova',
         'Viper',
         'Yoru']

agent_roles = {'Astra':'Controller',
              'Breach':'Initiator',
         'Brimstone':'Controller',
         'Chamber':'Sentinel',
         'Cypher':'Sentinel',
         'Fade':'Initiator',
         'Jett':'Duelist',
         'KAY/O':'Initiator',
         'Killjoy':'Sentinel',
         'Neon':'Duelist',
         'Omen':'Controller',
         'Phoenix':'Duelist',
         'Raze':'Duelist',
         'Reyna':'Duelist',
         'Sage':'Sentinel',
         'Skye':'Initiator',
         'Sova':'Initiator',
         'Viper':'Controller',
         'Yoru':'Duelist'}
roles_list = ['Controller', 'Initiator', 'Duelist', 'Sentinel']
all_maps_list = ['Ascent', 
                 'Bind', 
                 'Breeze', 
                 'Fracture', 
                 'Haven', 
                 'Icebox', 
                 'Pearl', 
                 'Split']
weapons = ['Vandal',
             'Phantom',
             'Spectre',
             'Ghost',
             'Classic',
             'Operator',
             'Sheriff',
             'Guardian',
             'Marshal',
             'Odin',
             'Judge',
             'Bulldog',
             'Stinger',
             'Frenzy',
             'Ares',
             'Shorty',
             'Bucky']

In [31]:
invalid_keys = [
    'currRank',
]

def extract_player_rows(game_metadata, round_summary, player_summary, player_rounds_kills):
    winning_team = get_winner(round_summary)
    game_datetime = game_metadata['dateStarted']
    map_pick = game_metadata['mapName']
    
    data = list()
    
    for i in player_summary:
        new_record = dict()
        new_record['name'] = i['attributes']['platformUserIdentifier']
        
        agent = i['metadata']['agentName']
        agent_role = agent_roles[agent]
        
        new_record['game_datetime'] = game_datetime
        new_record['won_game'] = int(winning_team == i['metadata']['teamId'])
        new_record['team'] = i['metadata']['teamId']
        new_record['map_pick'] = map_pick
        
        for j in i['stats'].keys():
            if j in invalid_keys:
                continue
            new_record[j] = i['stats'][j]['value']
        
        for j in roles_list:
            if j == agent_role:
                new_record[f'role_{j}'] = 1
            else:
                new_record[f'role_{j}'] = 0
        
        for j in all_agents_list:
            if j == agent:
                new_record[f'agent_{j}'] = 1
            else:
                new_record[f'agent_{j}'] = 0
        
        weapon_kills_dict = {j: 0 for j in weapons}
        for j in player_rounds_kills:
            if 'platformInfo' in j and i['attributes']['platformUserIdentifier'] != j['platformInfo']['platformUserHandle']:
                if j['metadata']['weaponName'] in weapons:
                    weapon_kills_dict[j['metadata']['weaponName']] += 1
            
        data.append(new_record)
            
    return data
        
        
processed_match_records = extract_player_rows(game_metadata, round_summary, player_summary, player_rounds_kills)
# processed_match_records[8]

In [32]:


def create_agent_row(agent_list, map_pick, game_win):

    
    for i in agent_list:
        if i not in all_agents_list:
            raise Exception(f'Invalid agent: {i}')
            
    if map_pick not in all_maps_list:
        raise Exception(f'Invalid map: {map_pick}')
    
    row = dict()
    
    roles_dict = {f'role_{i}': 0 for i in roles_list}
    
    for i in all_agents_list:
        row['agent_' + i] = int(i in agent_list)
                
    for i in agent_list:
        roles_dict[f'role_{agent_roles[i]}'] += 1
    
    roles_dict_keys = list(roles_dict.keys())
    
    for i in roles_dict_keys:
        for j in roles_dict_keys:
            if roles_list.index(i.split('_')[-1]) >= roles_list.index(j.split('_')[-1]):
                continue
            roles_dict[f'{i}*{j}'] = roles_dict[i] * roles_dict[j]
        
    for i in all_maps_list:
        row['map_' + i] = int(i == map_pick)
        
    row.update(roles_dict)
        
    row['game_win'] = game_win
    return row


def extract_team_rows(game_metadata, game_attributes, round_summary, player_summary):
    
    team_red_agents = list()
    team_blue_agents = list()
    
    map_pick = game_metadata['mapName']
        
    winning_team = get_winner(round_summary)
    
    for i in player_summary:
        if i['metadata']['teamId'] == 'Red':
            team_red_agents.append(i['metadata']['agentName'])
        if i['metadata']['teamId'] == 'Blue':
            team_blue_agents.append(i['metadata']['agentName'])
            
    row1 = create_agent_row(team_red_agents, map_pick, int('Red' == winning_team))
    row2 = create_agent_row(team_blue_agents, map_pick, int('Blue' == winning_team))
    
    return [row1, row2]
    
    




In [33]:
def get_all_processed_data(): 
    files = glob.glob(r'C:\Users\trist\OneDrive\Documents\game_data\valorant_raw/*.json')
    
    all_records = list()
    agent_records = list()
    
    for file in files:
        
        with open(file, 'r') as f:
            json_data = json.load(f)
        json_data = json.loads(json_data)
        if 'data' not in json_data:
            print(f'error {file}')
            continue
        game_metadata = json_data['data']['metadata']
        game_attributes = json_data['data']['attributes']

        player_rounds = [i for i in json_data['data']['segments'] if i['type'] == 'player-round']
        player_rounds_damage = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-damage']
        player_rounds_kills = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-kills']
        player_summary = [i for i in json_data['data']['segments'] if i['type'] == 'player-summary']
        round_summary = [i for i in json_data['data']['segments'] if i['type'] == 'round-summary']
        team_summary = [i for i in json_data['data']['segments'] if i['type'] == 'team-summary']
        
        all_records.extend(extract_player_rows(game_metadata, round_summary, player_summary, player_rounds_kills))
        agent_records.extend(extract_team_rows(game_metadata, game_attributes, round_summary, player_summary))
    return all_records, agent_records
        
all_records, all_agent_records = get_all_processed_data()
len(all_records), len(all_agent_records)


(24110, 4814)

In [34]:
user_record_count = dict()

for i in all_records:
    user_record_count.setdefault(i['name'], 0)
    user_record_count[i['name']] += 1

user_record_count_sorted = list()

for k, v in user_record_count.items():
    user_record_count_sorted.append({'user':k, 'count':v})

sorted(user_record_count_sorted, key = lambda x: x['count'], reverse = True)[:20]


[{'user': 'strawberry milk#우유갠디', 'count': 37},
 {'user': 'Thug Twerker#money', 'count': 30},
 {'user': 'IAmTempest#NA1', 'count': 26},
 {'user': 'Daddy Brim#OTP', 'count': 26},
 {'user': 'sussy#queer', 'count': 26},
 {'user': 'BishWhoDontMiss#6558', 'count': 26},
 {'user': 'YungCheddaBigD1k#2594', 'count': 25},
 {'user': 'Barbie#nails', 'count': 25},
 {'user': 'I bark for mommy#6969', 'count': 24},
 {'user': 'Tan#0008', 'count': 23},
 {'user': 'Arowana#queer', 'count': 23},
 {'user': 'DaddyMingze#NA1', 'count': 22},
 {'user': 'No Heads Hawes#2025', 'count': 22},
 {'user': 'milktea#dad', 'count': 22},
 {'user': 'ImLonelyPlease#1783', 'count': 22},
 {'user': 'Swift Raptor#NA1', 'count': 22},
 {'user': 'Yor#rice', 'count': 21},
 {'user': 'hardstuckiron#3570', 'count': 21},
 {'user': 'DarkPegasis123#NA1', 'count': 21},
 {'user': '32Equr#6990', 'count': 21}]

In [35]:
all_records = [i for i in all_records if user_record_count[i['name']] >= 5]
len(all_records)

4487

In [36]:
all_records_df = pd.DataFrame.from_dict(all_records)
all_records_df

,name,game_datetime,won_game,team,map_pick,rank,score,scorePerRound,killsPerRound,kills,...,agent_Neon,agent_Omen,agent_Phoenix,agent_Raze,agent_Reyna,agent_Sage,agent_Skye,agent_Sova,agent_Viper,agent_Yoru
0,Yor#rice,2022-07-08T06:53:09.588+00:00,1,Red,Breeze,Unranked,5623,234.291667,0.833333,20,...,0,0,0,0,0,0,1,0,0,0
1,hardstuckiron#3570,2022-08-07T22:08:58.634+00:00,1,Blue,Icebox,Bronze 1,6042,402.800000,1.466667,22,...,0,0,0,0,0,0,0,0,0,0
2,Andreww#1178,2022-08-07T22:08:58.634+00:00,1,Blue,Icebox,Iron 3,2032,135.466667,0.466667,7,...,0,0,1,0,0,0,0,0,0,0
3,Boldzero#101,2022-07-27T05:21:43.67+00:00,1,Red,Ascent,Bronze 3,1052,210.400000,0.800000,4,...,0,1,0,0,0,0,0,0,0,0
4,DarkPegasis123#NA1,2022-08-06T17:49:51.751+00:00,0,Blue,Pearl,Gold 3,2898,144.900000,0.400000,8,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4482,fork#9925,2022-08-02T06:46:59.463+00:00,1,Blue,Ascent,Bronze 1,2257,118.789474,0.421053,8,...,0,0,0,0,0,0,0,0,0,1
4483,Dark Dick#2499,2022-08-02T06:46:59.463+00:00,1,Blue,Ascent,Unranked,7557,397.736842,1.368421,26,...,0,0,0,0,1,0,0,0,0,0
4484,Ham#6386,2022-07-30T03:04:58.767+00:00,1,Red,Ascent,Bronze 1,5296,331.000000,1.125000,18,...,0,1,0,0,0,0,0,0,0,0
4485,Hitchman224#1988,2022-07-30T03:04:58.767+00:00,1,Red,Ascent,Unranked,3455,215.937500,0.750000,12,...,0,0,0,0,0,0,0,0,0,0


In [37]:
all_records_df['name'].nunique()

319

In [38]:
all_records_df['game_datetime'].min()

'2022-03-26T16:46:26.008+00:00'

In [39]:
all_records_df['map_pick'].value_counts(normalize=True)

Fracture    0.147537
Ascent      0.147092
Haven       0.145309
Breeze      0.141074
Bind        0.138846
Pearl       0.137285
Icebox      0.135503
Split       0.007355
Name: map_pick, dtype: float64

In [40]:
all_records_df['killsPerRound'].describe()

count    4487.000000
mean        0.805016
std         1.411051
min         0.000000
25%         0.500000
50%         0.681818
75%         0.900000
max        39.000000
Name: killsPerRound, dtype: float64

In [41]:
all_records_df['rank'].value_counts(normalize=True)

Bronze 1      0.131937
Bronze 2      0.121685
Bronze 3      0.111656
Silver 1      0.098953
Silver 3      0.090706
Iron 3        0.078895
Unranked      0.074437
Silver 2      0.061734
Gold 1        0.048585
Gold 3        0.043236
Iron 2        0.042567
Gold 2        0.041453
Platinum 1    0.026967
Platinum 2    0.011812
Platinum 3    0.008246
Iron 1        0.006463
Diamond 1     0.000669
Name: rank, dtype: float64

In [42]:

def get_player_features(player_df):
    player_df = player_df.sort_values(by = ['game_datetime'])

    data = list()


    row_counter = 0
    for n, (idx, row) in enumerate(player_df.iterrows()):
        if n < 5:
            continue

        past_game = player_df.iloc[n-1:n].mean().to_dict()
        past_5_games = player_df.iloc[n-5:n].mean().to_dict()

        new_x = dict()

#         for k, v in past_game.items():
#             new_x[f'past_game_{k}'] = v

        for k, v in past_5_games.items():
            new_x[f'past_5_games_avg_{k}'] = v


        new_x['future_won_game'] =  row['won_game']
        data.append(new_x)


    data_df = pd.DataFrame.from_dict(data)
    data_df = data_df.fillna(0)
    return data_df



In [43]:

training_players, val_players = train_test_split(list(set(all_records_df['name'].tolist())))

training_data_dfs = list()
val_data_dfs = list()
all_data_dfs= list()

for i in training_players:
    
    player_df = all_records_df[all_records_df['name'] == i]
    player_data_df = get_player_features(player_df)
    training_data_dfs.append(player_data_df)

    
for i in val_players:
    
    player_df = all_records_df[all_records_df['name'] == i]
    player_data_df = get_player_features(player_df)
    val_data_dfs.append(player_data_df)
    
for i in list(set(all_records_df['name'].tolist())):
    
    player_df = all_records_df[all_records_df['name'] == i]
    player_data_df = get_player_features(player_df)
    all_data_dfs.append(player_data_df)


    
training_data_df = pd.concat(training_data_dfs)
val_data_df = pd.concat(val_data_dfs)
all_data_df = pd.concat(all_data_dfs)

len(training_players), len(val_players)

(239, 80)

In [44]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# training_data_df.corr().sort_values(by=['future_won_game'])[['future_won_game']]

In [45]:
# training_data_df.isna().sum()

In [46]:
rf = RandomForestClassifier()

x = training_data_df.drop('future_won_game', axis = 1)
y = training_data_df['future_won_game']

x_val = val_data_df.drop('future_won_game', axis = 1)
y_val = val_data_df['future_won_game']

rf.fit(x, y)

preds = rf.predict(x_val)

accuracy_score(preds, y_val)

0.5302843016069221

In [47]:
x

,past_5_games_avg_won_game,past_5_games_avg_score,past_5_games_avg_scorePerRound,past_5_games_avg_killsPerRound,past_5_games_avg_kills,past_5_games_avg_deaths,past_5_games_avg_assists,past_5_games_avg_kdRatio,past_5_games_avg_damage,past_5_games_avg_damagePerRound,past_5_games_avg_singleKills,past_5_games_avg_doubleKills,past_5_games_avg_tripleKills,past_5_games_avg_quadraKills,past_5_games_avg_pentaKills,past_5_games_avg_multiKills,past_5_games_avg_grenadeCasts,past_5_games_avg_ability1Casts,past_5_games_avg_ability2Casts,past_5_games_avg_ultimateCasts,past_5_games_avg_grenadeCastsPerRound,past_5_games_avg_ability1CastsPerRound,past_5_games_avg_ability2CastsPerRound,past_5_games_avg_ultimateCastsPerRound,past_5_games_avg_plants,past_5_games_avg_defuses,past_5_games_avg_firstKills,past_5_games_avg_firstDeaths,past_5_games_avg_esr,past_5_games_avg_firstKillsPerRound,past_5_games_avg_firstDeathsPerRound,past_5_games_avg_econRating,past_5_games_avg_hsAccuracy,past_5_games_avg_kast,past_5_games_avg_role_Controller,past_5_games_avg_role_Initiator,past_5_games_avg_role_Duelist,past_5_games_avg_role_Sentinel,past_5_games_avg_agent_Astra,past_5_games_avg_agent_Breach,past_5_games_avg_agent_Brimstone,past_5_games_avg_agent_Chamber,past_5_games_avg_agent_Cypher,past_5_games_avg_agent_Fade,past_5_games_avg_agent_Jett,past_5_games_avg_agent_KAY/O,past_5_games_avg_agent_Killjoy,past_5_games_avg_agent_Neon,past_5_games_avg_agent_Omen,past_5_games_avg_agent_Phoenix,past_5_games_avg_agent_Raze,past_5_games_avg_agent_Reyna,past_5_games_avg_agent_Sage,past_5_games_avg_agent_Skye,past_5_games_avg_agent_Sova,past_5_games_avg_agent_Viper,past_5_games_avg_agent_Yoru
0,0.4,6268.8,259.988520,0.896458,21.6,16.8,5.2,1.299095,3928.6,163.159793,7.8,4.6,1.0,0.4,0.0,1.4,14.8,11.2,9.4,3.4,0.614959,0.466588,0.384646,0.141426,0.6,0.6,2.2,2.8,37.238095,0.089548,0.113383,69.6,10.980520,70.445678,0.0,0.0,0.4,0.6,0.0,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.2,0.0,0.0,0.0,0.0
1,0.2,6216.2,266.322863,0.895448,21.0,17.2,5.0,1.226275,3875.8,166.608278,7.6,4.0,1.0,0.6,0.0,1.6,14.0,11.0,10.8,3.4,0.600818,0.479719,0.474545,0.145466,0.4,0.4,2.2,3.4,37.238095,0.089548,0.148737,70.2,11.330761,64.789112,0.0,0.0,0.6,0.4,0.0,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.4,0.0,0.0,0.0,0.0,0.0,0.0
2,0.4,6270.0,297.546393,1.024407,21.6,15.4,4.4,1.504169,3925.2,186.691536,6.0,4.4,1.6,0.6,0.0,2.2,12.2,12.2,8.6,3.6,0.560094,0.591032,0.414364,0.169448,0.4,0.4,1.8,2.8,38.666667,0.078236,0.129732,79.6,11.652023,66.237076,0.0,0.0,0.6,0.4,0.0,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.4,5595.4,293.439799,1.080451,19.8,11.6,3.0,2.546275,3471.0,182.104722,4.4,4.6,1.4,0.6,0.0,2.0,10.0,11.4,7.6,3.2,0.538116,0.602021,0.375902,0.154064,0.0,0.0,1.4,2.0,32.000000,0.062852,0.098963,99.4,10.932420,91.072240,0.0,0.0,0.6,0.4,0.0,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.2,0.0,0.0,0.0,0.0,0.0,0.0
4,0.4,5341.4,284.694345,1.044483,18.8,11.4,3.4,2.499608,3336.2,177.755710,3.6,4.4,1.2,0.8,0.0,2.0,8.0,13.4,7.4,3.2,0.453926,0.694116,0.369578,0.155645,0.0,0.0,1.6,1.6,38.000000,0.073128,0.081967,99.4,10.292260,89.886470,0.0,0.0,0.8,0.2,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.2,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,0.4,4058.2,201.057714,0.702026,14.2,14.8,5.2,1.067908,2630.2,130.027366,5.2,2.6,1.4,0.0,0.0,1.4,4.2,1.6,14.8,1.8,0.191811,0.071196,0.718076,0.083090,0.2,0.2,1.4,2.2,33.333333,0.068815,0.104611,57.6,16.215007,64.368226,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0.6,4163.6,188.664445,0.672219,14.8,14.8,6.0,1.103203,2747.2,124.142750,6.4,2.0,1.6,0.0,0.0,1.6,4.8,1.4,15.2,2.4,0.210081,0.058696,0.685384,0.106167,0.2,0.2,1.8,2.2,38.333333,0.079392,0.099803,58.6,15.551786,69.752

In [48]:
fi_list = list()
for i, j in zip(x_val.columns, rf.feature_importances_):
    fi_list.append({'column':i, 'importance':j})
    
pd.DataFrame.from_dict(fi_list).sort_values(by = ['importance'])[-10:]

,column,importance
8,past_5_games_avg_damage,0.031528
18,past_5_games_avg_ability2Casts,0.031666
29,past_5_games_avg_firstKillsPerRound,0.032399
33,past_5_games_avg_kast,0.032963
31,past_5_games_avg_econRating,0.033054
22,past_5_games_avg_ability2CastsPerRound,0.033323
20,past_5_games_avg_grenadeCastsPerRound,0.033818
23,past_5_games_avg_ultimateCastsPerRound,0.033932
16,past_5_games_avg_grenadeCasts,0.034172
32,past_5_games_avg_hsAccuracy,0.035873


In [49]:
rf = LogisticRegression()

x = training_data_df.drop('future_won_game', axis = 1)
y = training_data_df['future_won_game']

x_val = val_data_df.drop('future_won_game', axis = 1)
y_val = val_data_df['future_won_game']

rf.fit(x, y)

preds = rf.predict(x_val)

accuracy_score(preds, y_val)

C:\Users\trist\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5352286773794809

In [50]:

df_interaction = pd.DataFrame()

df_interaction['future_won_game'] = all_data_df['future_won_game']

columns_list = training_data_df.columns.tolist()

for i in columns_list:
    df_interaction[i] =  all_data_df[i]
    for j in columns_list:
        if  i == 'future_won_game' or j == 'future_won_game':
            continue
        if columns_list.index(i) >=columns_list.index(j):
            continue
        df_interaction[f'{i}_mul_{j}'] = all_data_df[i]*all_data_df[j]
        
df_interaction.head()



,future_won_game,past_5_games_avg_won_game,past_5_games_avg_won_game_mul_past_5_games_avg_score,past_5_games_avg_won_game_mul_past_5_games_avg_scorePerRound,past_5_games_avg_won_game_mul_past_5_games_avg_killsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_kills,past_5_games_avg_won_game_mul_past_5_games_avg_deaths,past_5_games_avg_won_game_mul_past_5_games_avg_assists,past_5_games_avg_won_game_mul_past_5_games_avg_kdRatio,past_5_games_avg_won_game_mul_past_5_games_avg_damage,past_5_games_avg_won_game_mul_past_5_games_avg_damagePerRound,past_5_games_avg_won_game_mul_past_5_games_avg_singleKills,past_5_games_avg_won_game_mul_past_5_games_avg_doubleKills,past_5_games_avg_won_game_mul_past_5_games_avg_tripleKills,past_5_games_avg_won_game_mul_past_5_games_avg_quadraKills,past_5_games_avg_won_game_mul_past_5_games_avg_pentaKills,past_5_games_avg_won_game_mul_past_5_games_avg_multiKills,past_5_games_avg_won_game_mul_past_5_games_avg_grenadeCasts,past_5_games_avg_won_game_mul_past_5_games_avg_ability1Casts,past_5_games_avg_won_game_mul_past_5_games_avg_ability2Casts,past_5_games_avg_won_game_mul_past_5_games_avg_ultimateCasts,past_5_games_avg_won_game_mul_past_5_games_avg_grenadeCastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_ability1CastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_ability2CastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_ultimateCastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_plants,past_5_games_avg_won_game_mul_past_5_games_avg_defuses,past_5_games_avg_won_game_mul_past_5_games_avg_firstKills,past_5_games_avg_won_game_mul_past_5_games_avg_firstDeaths,past_5_games_avg_won_game_mul_past_5_games_avg_esr,past_5_games_avg_won_game_mul_past_5_games_avg_firstKillsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_firstDeathsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_econRating,past_5_games_avg_won_game_mul_past_5_games_avg_hsAccuracy,past_5_games_avg_won_game_mul_past_5_games_avg_kast,past_5_games_avg_won_game_mul_past_5_games_avg_role_Controller,past_5_games_avg_won_game_mul_past_5_games_avg_role_Initiator,past_5_games_avg_won_game_mul_past_5_games_avg_role_Duelist,past_5_games_avg_won_game_mul_past_5_games_avg_role_Sentinel,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Astra,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Breach,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Brimstone,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Chamber,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Cypher,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Fade,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Jett,past_5_games_avg_won_game_mul_past_5_games_avg_agent_KAY/O,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Killjoy,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Neon,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Omen,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Phoenix,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Raze,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Reyna,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Sage,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Skye,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Sova,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Viper,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Yoru,past_5_games_avg_score,past_5_games_avg_score_mul_past_5_games_avg_scorePerRound,past_5_games_avg_score_mul_past_5_games_avg_killsPerRound,past_5_games_avg_score_mul_past_5_games_avg_kills,past_5_games_avg_score_mul_past_5_games_avg_deaths,past_5_games_avg_score_mul_past_5_games_avg_assists,past_5_games_avg_score_mul_past_5_games_avg_kdRatio,past_5_games_avg_score_mul_past_5_games_avg_damage,past_5_games_avg_score_mul_past_5_games_avg_damagePerRound,past_5_games_avg_score_mul_past_5_games_avg_singleKills,past_5_games_avg_score_mul_past_5_games_avg_doubleKills,past_5_games_avg_score_mul_past_5_games_avg_tripleKills,past_5_ga

In [51]:
pd.set_option('max_colwidth', 400)



fi_list = list()

for i in df_interaction.columns:
    
    if i == 'future_won_game':
        continue
        
    rel = linregress(df_interaction['future_won_game'], df_interaction[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False]).head(20)

,column,slope,intercept,rvalue,r2,pvalue,stderr
352,past_5_games_avg_assists_mul_past_5_games_avg_econRating,24.172596,267.868998,0.100416,0.010083,6.251012e-08,4.455222
328,past_5_games_avg_assists_mul_past_5_games_avg_kdRatio,0.535688,5.071674,0.093659,0.008772,4.519629e-07,0.105925
330,past_5_games_avg_assists_mul_past_5_games_avg_damagePerRound,50.111209,629.607265,0.090580,0.008205,1.065871e-06,10.248615
329,past_5_games_avg_assists_mul_past_5_games_avg_damage,1044.416001,12864.466362,0.083866,0.007033,6.299693e-06,230.838485
381,past_5_games_avg_kdRatio_mul_past_5_games_avg_singleKills,0.520398,6.354227,0.080894,0.006544,1.327587e-05,0.119274
379,past_5_games_avg_kdRatio_mul_past_5_games_avg_damage,377.671802,3094.174410,0.080890,0.006543,1.328773e-05,86.565445
453,past_5_games_avg_damage_mul_past_5_games_avg_kast,11615.184648,191242.986941,0.080854,0.006537,1.340822e-05,2663.508079
382,past_5_games_avg_kdRatio_mul_past_5_games_avg_doubleKills,0.377199,2.941858,0.080661,0.006506,1.406062e-05,0.086705
402,past_5_games_avg_kdRatio_mul_past_5_games_avg_econRating,10.163142,67.644406,0.080297,0.006448,1.537238e-05,2.346789
349,past_5_games_avg_assists_mul_past_5_games_avg_esr,38.439843,284.450815,0.079590,0.006334,1.826905e-05,8.955646


In [52]:
fi_list = list()

for i in all_data_df.columns:
    if i == 'future_won_game':
        continue
    rel = linregress(all_data_df['future_won_game'], all_data_df[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False])

,column,slope,intercept,rvalue,r2,pvalue,stderr
7,past_5_games_avg_kdRatio,0.082228,1.078237,0.078673,6.189510e-03,0.000023,0.019382
8,past_5_games_avg_damage,122.478333,2663.122663,0.074771,5.590735e-03,0.000057,30.384946
9,past_5_games_avg_damagePerRound,5.290700,132.663381,0.072619,5.273544e-03,0.000093,1.351652
31,past_5_games_avg_econRating,2.451660,57.375387,0.068439,4.683929e-03,0.000230,0.664793
6,past_5_games_avg_assists,0.223172,4.726295,0.067118,4.504839e-03,0.000304,0.061712
2,past_5_games_avg_scorePerRound,-50.443827,260.797047,-0.064048,4.102105e-03,0.000568,14.620531
11,past_5_games_avg_doubleKills,0.128621,2.489173,0.062952,3.962909e-03,0.000706,0.037931
3,past_5_games_avg_killsPerRound,-0.157005,0.880484,-0.062817,3.945915e-03,0.000725,0.046401
28,past_5_games_avg_esr,5.507956,60.389841,0.062225,3.871930e-03,0.000814,1.643370
30,past_5_games_avg_firstDeathsPerRound,-0.004875,0.098077,-0.053849,2.899691e-03,0.003771,0.001682



## Agent pick prediction data set creation

In [53]:
all_records, all_agent_records = get_all_processed_data()


In [54]:
# agents = list()

# for i in all_records:
#     agents.append(i['agent'])
    
# sorted(list(set(agents)))

In [55]:
maps = list()

for i in all_records:
    maps.append(i['map_pick'])
    
sorted(list(set(maps)))

['Ascent', 'Bind', 'Breeze', 'Fracture', 'Haven', 'Icebox', 'Pearl', 'Split']

In [56]:
all_agent_records_df = pd.DataFrame.from_dict(all_agent_records)
all_agent_records_df.head()

,agent_Astra,agent_Breach,agent_Brimstone,agent_Chamber,agent_Cypher,agent_Fade,agent_Jett,agent_KAY/O,agent_Killjoy,agent_Neon,agent_Omen,agent_Phoenix,agent_Raze,agent_Reyna,agent_Sage,agent_Skye,agent_Sova,agent_Viper,agent_Yoru,map_Ascent,map_Bind,map_Breeze,map_Fracture,map_Haven,map_Icebox,map_Pearl,map_Split,role_Controller,role_Initiator,role_Duelist,role_Sentinel,role_Controller*role_Initiator,role_Controller*role_Duelist,role_Controller*role_Sentinel,role_Initiator*role_Duelist,role_Initiator*role_Sentinel,role_Duelist*role_Sentinel,game_win
0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,1,2,1,1,2,1,2,1,2,1
1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,1,2,1,1,2,1,2,1,2,0
2,0,0,0,1,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,2,2,0,0,0,2,2,4,0
3,0,0,0,1,0,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2,3,0,0,0,0,0,6,1
4,0,0,0,1,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,2,2,0,2,2,0,0,4,1


In [57]:
train_df, val_df = train_test_split(all_agent_records_df)

train_x = train_df.drop('game_win', axis = 1)
train_y = train_df['game_win']

val_x = val_df.drop('game_win', axis = 1)
val_y = val_df['game_win']
train_x.head()

,agent_Astra,agent_Breach,agent_Brimstone,agent_Chamber,agent_Cypher,agent_Fade,agent_Jett,agent_KAY/O,agent_Killjoy,agent_Neon,agent_Omen,agent_Phoenix,agent_Raze,agent_Reyna,agent_Sage,agent_Skye,agent_Sova,agent_Viper,agent_Yoru,map_Ascent,map_Bind,map_Breeze,map_Fracture,map_Haven,map_Icebox,map_Pearl,map_Split,role_Controller,role_Initiator,role_Duelist,role_Sentinel,role_Controller*role_Initiator,role_Controller*role_Duelist,role_Controller*role_Sentinel,role_Initiator*role_Duelist,role_Initiator*role_Sentinel,role_Duelist*role_Sentinel
4199,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,1,1,2,1,1,2,1,2,1,2
171,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,1,0,0,0,1,2,1,1,2,1,1,2,2,1
3610,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,2,1,1,2,1,2,1,2
4416,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,3,0,2,0,0,6,0,0,0,0
694,0,0,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,1,1,2,1,1,2,1,2,2


In [58]:
rf = RandomForestClassifier()

rf.fit(train_x, train_y)

preds = rf.predict(val_x)

accuracy_score(preds, val_y)

0.5091059602649006

In [59]:
fi_list = list()
for i, j in zip(train_x.columns, rf.feature_importances_):
    fi_list.append({'column':i, 'importance':j})
    
pd.DataFrame.from_dict(fi_list).sort_values(by = ['importance'])[-10:]

,column,importance
22,map_Fracture,0.033283
24,map_Icebox,0.033558
18,agent_Yoru,0.035814
10,agent_Omen,0.036882
14,agent_Sage,0.038454
3,agent_Chamber,0.039598
11,agent_Phoenix,0.040925
13,agent_Reyna,0.042284
12,agent_Raze,0.047749
6,agent_Jett,0.048467


In [60]:
rf = LogisticRegression()

rf.fit(train_x, train_y)

preds = rf.predict(val_x)

accuracy_score(preds, val_y)

C:\Users\trist\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5240066225165563

In [61]:
fi_list = list()

for i in train_x.columns:
    rel = linregress(train_y, train_x[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False])

,column,slope,intercept,rvalue,r2,pvalue,stderr
9,agent_Neon,-0.038030,0.228715,-0.046719,2.182686e-03,0.004919,0.013514
5,agent_Fade,0.034974,0.184752,0.043523,1.894269e-03,0.008801,0.013343
36,role_Duelist*role_Sentinel,0.093682,2.215359,0.035960,1.293103e-03,0.030455,0.043272
13,agent_Reyna,0.035076,0.570952,0.035639,1.270144e-03,0.031968,0.016347
7,agent_KAY/O,-0.025316,0.221480,-0.031146,9.700595e-04,0.060895,0.013503
17,agent_Viper,0.024378,0.268225,0.027131,7.360989e-04,0.102559,0.014928
16,agent_Sova,-0.018870,0.194213,-0.024313,5.911127e-04,0.143486,0.012896
25,map_Pearl,-0.014814,0.144129,-0.021563,4.649697e-04,0.194480,0.011416
29,role_Duelist,0.030756,1.852532,0.021384,4.572927e-04,0.198205,0.023899
14,agent_Sage,0.020923,0.564830,0.021164,4.479310e-04,0.202862,0.016427


In [127]:
train_df, val_df = train_test_split(all_agent_records_df)


df_interaction_train = pd.DataFrame()
df_interaction_val = pd.DataFrame()

df_interaction_train['game_win'] = train_df['game_win']
df_interaction_val['game_win'] = val_df['game_win']

columns_list = train_df.columns.tolist()

for i in columns_list:
    df_interaction_train[i] =  train_df[i]
    df_interaction_val[i] =  val_df[i]
    for j in columns_list:
        if  i == 'game_win' or j == 'game_win':
            continue
        if columns_list.index(i) >=columns_list.index(j):
            continue
        df_interaction_train[f'{i}_mul_{j}'] = train_df[i]*train_df[j]
        df_interaction_val[f'{i}_mul_{j}'] = val_df[i]*val_df[j]
        
df_interaction_train.head()

,game_win,agent_Astra,agent_Astra_mul_agent_Breach,agent_Astra_mul_agent_Brimstone,agent_Astra_mul_agent_Chamber,agent_Astra_mul_agent_Cypher,agent_Astra_mul_agent_Fade,agent_Astra_mul_agent_Jett,agent_Astra_mul_agent_KAY/O,agent_Astra_mul_agent_Killjoy,agent_Astra_mul_agent_Neon,agent_Astra_mul_agent_Omen,agent_Astra_mul_agent_Phoenix,agent_Astra_mul_agent_Raze,agent_Astra_mul_agent_Reyna,agent_Astra_mul_agent_Sage,agent_Astra_mul_agent_Skye,agent_Astra_mul_agent_Sova,agent_Astra_mul_agent_Viper,agent_Astra_mul_agent_Yoru,agent_Astra_mul_map_Ascent,agent_Astra_mul_map_Bind,agent_Astra_mul_map_Breeze,agent_Astra_mul_map_Fracture,agent_Astra_mul_map_Haven,agent_Astra_mul_map_Icebox,agent_Astra_mul_map_Pearl,agent_Astra_mul_map_Split,agent_Astra_mul_role_Controller,agent_Astra_mul_role_Initiator,agent_Astra_mul_role_Duelist,agent_Astra_mul_role_Sentinel,agent_Astra_mul_role_Controller*role_Initiator,agent_Astra_mul_role_Controller*role_Duelist,agent_Astra_mul_role_Controller*role_Sentinel,agent_Astra_mul_role_Initiator*role_Duelist,agent_Astra_mul_role_Initiator*role_Sentinel,agent_Astra_mul_role_Duelist*role_Sentinel,agent_Breach,agent_Breach_mul_agent_Brimstone,agent_Breach_mul_agent_Chamber,agent_Breach_mul_agent_Cypher,agent_Breach_mul_agent_Fade,agent_Breach_mul_agent_Jett,agent_Breach_mul_agent_KAY/O,agent_Breach_mul_agent_Killjoy,agent_Breach_mul_agent_Neon,agent_Breach_mul_agent_Omen,agent_Breach_mul_agent_Phoenix,agent_Breach_mul_agent_Raze,agent_Breach_mul_agent_Reyna,agent_Breach_mul_agent_Sage,agent_Breach_mul_agent_Skye,agent_Breach_mul_agent_Sova,agent_Breach_mul_agent_Viper,agent_Breach_mul_agent_Yoru,agent_Breach_mul_map_Ascent,agent_Breach_mul_map_Bind,agent_Breach_mul_map_Breeze,agent_Breach_mul_map_Fracture,agent_Breach_mul_map_Haven,agent_Breach_mul_map_Icebox,agent_Breach_mul_map_Pearl,agent_Breach_mul_map_Split,agent_Breach_mul_role_Controller,agent_Breach_mul_role_Initiator,agent_Breach_mul_role_Duelist,agent_Breach_mul_role_Sentinel,agent_Breach_mul_role_Controller*role_Initiator,agent_Breach_mul_role_Controller*role_Duelist,agent_Breach_mul_role_Controller*role_Sentinel,agent_Breach_mul_role_Initiator*role_Duelist,agent_Breach_mul_role_Initiator*role_Sentinel,agent_Breach_mul_role_Duelist*role_Sentinel,agent_Brimstone,agent_Brimstone_mul_agent_Chamber,agent_Brimstone_mul_agent_Cypher,agent_Brimstone_mul_agent_Fade,agent_Brimstone_mul_agent_Jett,agent_Brimstone_mul_agent_KAY/O,agent_Brimstone_mul_agent_Killjoy,agent_Brimstone_mul_agent_Neon,agent_Brimstone_mul_agent_Omen,agent_Brimstone_mul_agent_Phoenix,agent_Brimstone_mul_agent_Raze,agent_Brimstone_mul_agent_Reyna,agent_Brimstone_mul_agent_Sage,agent_Brimstone_mul_agent_Skye,agent_Brimstone_mul_agent_Sova,agent_Brimstone_mul_agent_Viper,agent_Brimstone_mul_agent_Yoru,agent_Brimstone_mul_map_Ascent,agent_Brimstone_mul_map_Bind,agent_Brimstone_mul_map_Breeze,agent_Brimstone_mul_map_Fracture,agent_Brimstone_mul_map_Haven,agent_Brimstone_mul_map_Icebox,agent_Brimstone_mul_map_Pearl,agent_Brimstone_mul_map_Split,agent_Brimstone_mul_role_Controller,agent_Brimstone_mul_role_Initiator,agent_Brimstone_mul_role_Duelist,agent_Brimstone_mul_role_Sentinel,agent_Brimstone_mul_role_Controller*role_Initiator,agent_Brimstone_mul_role_Controller*role_Duelist,agent_Brimstone_mul_role_Controller*role_Sentinel,agent_Brimstone_mul_role_Initiator*role_Duelist,agent_Brimstone_mul_role_Initiator*role_Sentinel,agent_Brimstone_mul_role_Duelist*role_Sentinel,agent_Chamber,agent_Chamber_mul_agent_Cypher,agent_Chamber_mul_agent_Fade,agent_Chamber_mul_agent_Jett,agent_Chamber_mul_agent_KAY/O,agent_Chamber_mul_agent_Killjoy,agent_Chamber_mul_agent_Neon,agent_Chamber_mul_agent_Omen,agent_Chamber_mul_agent_Phoenix,agent_Chamber_mul_agent_Raze,agent_Chamber_mul_agent_Reyna,agent_Chamber_mul_agent_Sage,agent_Chamber_mul_agent_Skye,agent_Chamber_mul_agent_Sova,agent_Chamber_mul_agent_Viper,agent_Chamber_mul_agent_Yoru,agent_Chamber_mul_map_Ascent,agent_Chamber_mul_map_Bind,agent_Chamber_mu

In [128]:
df_interaction_train.shape, df_interaction_val.shape

((3622, 704), (1208, 704))

In [144]:
rf = RandomForestClassifier(n_estimators=100, max_features=.5)

rf.fit( df_interaction_train.drop('game_win', axis = 1), df_interaction_train['game_win'])

preds = rf.predict(df_interaction_val.drop('game_win', axis = 1))

accuracy_score(preds, df_interaction_val['game_win'])

0.5248344370860927

In [129]:
rf = LogisticRegression()

rf.fit( df_interaction_train.drop('game_win', axis = 1), df_interaction_train['game_win'])

preds = rf.predict(df_interaction_val.drop('game_win', axis = 1))

accuracy_score(preds, df_interaction_val['game_win'])

C:\Users\trist\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.49751655629139074

In [140]:
rf = LogisticRegression()
pca = PCA(n_components=50)

x_pca = pca.fit_transform(df_interaction_train.drop('game_win', axis = 1))
x_val_pca = pca.transform(df_interaction_val.drop('game_win', axis = 1))

rf.fit(x_pca, df_interaction_train['game_win'])

preds = rf.predict(x_val_pca)

accuracy_score(preds, df_interaction_val['game_win'])

0.5140728476821192

In [145]:
rf = RandomForestClassifier(n_estimators=100, max_features=.5)
pca = PCA(n_components=50)

x_pca = pca.fit_transform(df_interaction_train.drop('game_win', axis = 1))
x_val_pca = pca.transform(df_interaction_val.drop('game_win', axis = 1))

rf.fit(x_pca, df_interaction_train['game_win'])

preds = rf.predict(x_val_pca)

accuracy_score(preds, df_interaction_val['game_win'])

0.5248344370860927

In [233]:

# for n_components in range(1, 100):
#     rf = SVC()
#     pca = PCA(n_components=n_components)

#     x_pca = pca.fit_transform(df_interaction_train.drop('game_win', axis = 1))
#     x_val_pca = pca.transform(df_interaction_val.drop('game_win', axis = 1))

#     rf.fit(x_pca, df_interaction_train['game_win'])

#     preds = rf.predict(x_val_pca)

#     print(n_components, accuracy_score(preds, df_interaction_val['game_win']))



In [112]:
fi_list = list()

for i in df_interaction_train.columns:
    if i == 'game_win':
        continue
    rel = linregress(df_interaction_train['game_win'], df_interaction_train[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False]).head(50)

,column,slope,intercept,rvalue,r2,pvalue,stderr
304,agent_Neon_mul_agent_Sova,-0.020870,0.044653,-0.057386,0.003293,0.000550,0.006034
58,agent_Breach_mul_map_Fracture,0.017197,0.015436,0.056159,0.003154,0.000721,0.005082
122,agent_Chamber_mul_agent_Viper,0.030746,0.098126,0.048469,0.002349,0.003526,0.010531
506,agent_Viper_mul_role_Sentinel,0.057420,0.302646,0.045943,0.002111,0.005684,0.020751
21,agent_Astra_mul_map_Breeze,0.004980,0.000551,0.045249,0.002047,0.006456,0.001827
72,agent_Breach_mul_role_Duelist*role_Sentinel,0.047929,0.109151,0.044768,0.002004,0.007045,0.017776
280,agent_Killjoy_mul_map_Bind,0.013333,0.017641,0.043298,0.001875,0.009157,0.005113
509,agent_Viper_mul_role_Controller*role_Sentinel,0.063649,0.346196,0.043201,0.001866,0.009315,0.024465
52,agent_Breach_mul_agent_Sova,-0.007703,0.012128,-0.042497,0.001806,0.010532,0.003010
200,agent_Fade_mul_role_Sentinel,0.043795,0.196803,0.042252,0.001785,0.010987,0.017212


In [62]:
from sklearn.decomposition import PCA

rf = LogisticRegression()
pca = PCA()

x_pca = pca.fit_transform(train_x)
x_val_pca = pca.transform(val_x)

rf.fit(x_pca, train_y)

preds = rf.predict(x_val_pca)

accuracy_score(preds, val_y)

0.5231788079470199

In [152]:

# from sklearn.decomposition import PCA

# rf = SVC()
# pca = PCA()

# x_pca = pca.fit_transform(train_x)
# x_val_pca = pca.transform(val_x)

# rf.fit(x_pca, train_y)

# preds = rf.predict(x_val_pca)

# accuracy_score(preds, val_y)

In [63]:
model= RandomForestClassifier(n_estimators=100, max_features = .9, max_samples=.5)

model.fit(train_x, train_y)

preds = model.predict(val_x)

accuracy_score(preds, val_y)

0.49503311258278143

In [64]:
# model.predict(val_x)

In [65]:
# model.predict_proba(val_x)[0][-1]

In [99]:
model= RandomForestClassifier(n_estimators=20, max_features = .5)

model.fit(train_x, train_y)

preds = model.predict(val_x)

accuracy_score(preds, val_y)

0.4908940397350993

In [100]:
model= RandomForestClassifier(n_estimators=20, max_features = .5)


train_df, val_df = train_test_split(all_agent_records_df)

all_x = all_agent_records_df.drop('game_win', axis = 1)
all_y = all_agent_records_df['game_win']

model.fit(all_x, all_y)


RandomForestClassifier(max_features=0.5, n_estimators=20)

In [159]:
def create_interactions(df: pd.DataFrame) -> pd.DataFrame:
    df_interaction = pd.DataFrame()
    
    if 'game_win' in df.columns:
        df_interaction['game_win'] = df['game_win']

    columns_list = sorted(df.columns.tolist())

    for i in columns_list:
        df_interaction[i] = df[i]
        for j in columns_list:
            if  i == 'game_win' or j == 'game_win':
                continue
            if columns_list.index(i) >=columns_list.index(j):
                continue
            df_interaction[f'{i}_mul_{j}'] = df[i]*df[j]
    return df_interaction

In [166]:
train_df, val_df = train_test_split(all_agent_records_df)


model = SVC(probability=True)
pca = PCA(n_components=12)

df_train = create_interactions(train_df)
df_val = create_interactions(val_df)

x_train_pca = pca.fit_transform(df_train.drop('game_win', axis = 1))
x_val_pca = pca.fit_transform(df_val.drop('game_win', axis = 1))

model.fit(x_train_pca, df_train['game_win'])

preds = model.predict(x_val_pca)
accuracy_score(preds, val_y)

0.5173841059602649

In [186]:
from sklearn.linear_model import LogisticRegression

train_df, val_df = train_test_split(all_agent_records_df)

model = LogisticRegression()
pca = PCA(n_components=12)

df_train = create_interactions(train_df)
df_val = create_interactions(val_df)

x_train_pca = pca.fit_transform(df_train.drop('game_win', axis = 1))
x_val_pca = pca.fit_transform(df_val.drop('game_win', axis = 1))

model.fit(x_train_pca, df_train['game_win'])

preds = model.predict(x_val_pca)
accuracy_score(preds, val_df['game_win'])
# preds

0.5074503311258278

In [ ]:
train_df, val_df = train_test_split(all_agent_records_df)


In [235]:
import random
results = list()

counter = 0

while True:
    counter += 1
    if counter > 10000:
        break
    i = random.randint(10, 50)
    j = random.randint(10, 99) * .01
    k = random.randint(1, 100)
    max_depth = random.randint(2, 10)
    

    model = RandomForestClassifier(n_estimators=i, max_features = j, max_depth=max_depth)
    pca = PCA(n_components=k)

    df_train = create_interactions(train_df)
    df_val = create_interactions(val_df)

    x_train_pca = pca.fit_transform(df_train.drop('game_win', axis = 1))
    x_val_pca = pca.fit_transform(df_val.drop('game_win', axis = 1))

    model.fit(x_train_pca, df_train['game_win'])

    preds = model.predict(x_val_pca)
    score= accuracy_score(preds, val_df['game_win'])

    results.append({'n_estimators':i,
                   'max_features':j,
                   'n_components':k,
                    'max_depth':max_depth,
                   'score':score})
    if score >= sorted(results, key = lambda x: x['score'], reverse = True)[0]['score']:
        print(counter, i, j, k, score, sorted(results, key = lambda x: x['score'], reverse = True)[0])
        
            
sorted(results, key = lambda x: x['score'], reverse = True)[:10]

1 30 0.75 72 0.49834437086092714 {'n_estimators': 30, 'max_features': 0.75, 'n_components': 72, 'max_depth': 10, 'score': 0.49834437086092714}
2 37 0.3 31 0.5057947019867549 {'n_estimators': 37, 'max_features': 0.3, 'n_components': 31, 'max_depth': 8, 'score': 0.5057947019867549}
3 11 0.85 15 0.5082781456953642 {'n_estimators': 11, 'max_features': 0.85, 'n_components': 15, 'max_depth': 8, 'score': 0.5082781456953642}
4 43 0.88 4 0.5347682119205298 {'n_estimators': 43, 'max_features': 0.88, 'n_components': 4, 'max_depth': 5, 'score': 0.5347682119205298}
139 42 0.18 36 0.5347682119205298 {'n_estimators': 43, 'max_features': 0.88, 'n_components': 4, 'max_depth': 5, 'score': 0.5347682119205298}
176 21 0.38 40 0.5364238410596026 {'n_estimators': 21, 'max_features': 0.38, 'n_components': 40, 'max_depth': 7, 'score': 0.5364238410596026}
275 19 0.9500000000000001 35 0.5372516556291391 {'n_estimators': 19, 'max_features': 0.9500000000000001, 'n_components': 35, 'max_depth': 5, 'score': 0.537251

KeyboardInterrupt: 

In [ ]:
# sorted(results, key = lambda x: x['score'], reverse = True)

In [205]:
from sklearn.linear_model import LogisticRegression

train_df, val_df = train_test_split(all_agent_records_df)

model = RandomForestClassifier(n_estimators=12, max_features = 0.25, max_depth=5)
pca = PCA(n_components=45)

df_train = create_interactions(train_df)
df_val = create_interactions(val_df)

x_train_pca = pca.fit_transform(df_train.drop('game_win', axis = 1))
x_val_pca = pca.fit_transform(df_val.drop('game_win', axis = 1))

model.fit(x_train_pca, df_train['game_win'])

preds = model.predict(x_val_pca)
accuracy_score(preds, val_df['game_win'])


0.5256622516556292

In [207]:
model = RandomForestClassifier(n_estimators=12, max_features = 0.25, max_depth=5)
pca = PCA(n_components=45)

df_all = create_interactions(all_agent_records_df)

x_pca = pca.fit_transform(df_all.drop('game_win', axis = 1))

model.fit(x_pca, df_all['game_win'])

preds = model.predict(x_pca)
preds.shape

(4830,)

In [208]:
def predict_best_lineup(model, vectorizer, map_pick, current_agent_list):
    
    agent_score = list()
    
    agent_tuples = list()
    
    for i1 in all_agents_list:
        for i2 in all_agents_list:
            for i3 in all_agents_list:
                for i4 in all_agents_list:
                    for i5 in all_agents_list:
                        if len(set([i1, i2, i3, i4, i5])) < 5:
                            continue
                        else:
                            agent_tuples.append(tuple(sorted([i1, i2, i3, i4, i5])))
    agent_tuples = list(set(agent_tuples))
    matched_agent_tuples = list()
    
    for i in agent_tuples:
        match = True
        for j in current_agent_list:
            if j not in i:
                match = False
        if match:
            matched_agent_tuples.append(i)
    
    inputs = list()
    features = list()
    
    for i in matched_agent_tuples:
        inputs.append({'agent1_name': i[0], 'agent2_name': i[1], 'agent3_name': i[2], 'agent4_name': i[3], 'agent5_name': i[4]})
        features.append(create_agent_row(list(i), map_pick, None))
    
    features_df = pd.DataFrame.from_dict(features)
    features_df_interactions = create_interactions(features_df)
    features_df_interactions_pca = pca.fit_transform(features_df_interactions.drop('game_win', axis = 1))
    
    inputs_df = pd.DataFrame.from_dict(inputs)
    
    inputs_df.index = features_df.index

    inputs_df['win_prob'] = model.predict_proba(features_df_interactions_pca)[:,-1]
    return inputs_df.sort_values('win_prob', ascending = False)


map_pick = 'Icebox'
current_agent_list = ['Phoenix',
                     'Reyna',
                      'Raze',
                     ]

predict_best_lineup(model,pca, map_pick, current_agent_list)

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
107,Brimstone,Chamber,Phoenix,Raze,Reyna,0.607421
67,Phoenix,Raze,Reyna,Sova,Yoru,0.581000
59,Omen,Phoenix,Raze,Reyna,Skye,0.571810
80,Jett,Phoenix,Raze,Reyna,Sova,0.570917
118,Breach,KAY/O,Phoenix,Raze,Reyna,0.555484
53,Fade,Phoenix,Raze,Reyna,Sage,0.555347
47,Cypher,Fade,Phoenix,Raze,Reyna,0.551490
15,Fade,Killjoy,Phoenix,Raze,Reyna,0.546094
38,Brimstone,Phoenix,Raze,Reyna,Skye,0.542216
11,Phoenix,Raze,Reyna,Sage,Skye,0.540944


In [209]:
['Ascent', 'Bind', 'Breeze', 'Fracture', 'Haven', 'Icebox', 'Pearl', 'Split']

['Ascent', 'Bind', 'Breeze', 'Fracture', 'Haven', 'Icebox', 'Pearl', 'Split']

In [216]:
map_pick = 'Ascent'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
8232,Breach,Neon,Phoenix,Skye,Yoru,0.671427
2486,Chamber,Cypher,Killjoy,Phoenix,Raze,0.647445
8463,Cypher,Killjoy,Neon,Phoenix,Sage,0.646556
3548,Breach,Phoenix,Reyna,Skye,Yoru,0.641796
6687,Chamber,Killjoy,Neon,Phoenix,Sage,0.640568


In [217]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
5030,Brimstone,Omen,Phoenix,Raze,Yoru,0.282677
4737,Chamber,Jett,Killjoy,Phoenix,Yoru,0.278072
7495,Cypher,Jett,Phoenix,Sage,Yoru,0.277034
107,Neon,Raze,Reyna,Sova,Yoru,0.273807
6787,Jett,Killjoy,Phoenix,Sage,Yoru,0.271779


In [218]:
map_pick = 'Bind'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
689,Breach,Fade,Phoenix,Reyna,Skye,0.661028
3453,Breach,Fade,Jett,Raze,Sova,0.657506
2847,Breach,Fade,Jett,Skye,Yoru,0.655379
2978,Breach,Fade,Neon,Raze,Skye,0.650395
3859,Jett,KAY/O,Neon,Skye,Yoru,0.645460


In [219]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
5390,Astra,Brimstone,Jett,Phoenix,Reyna,0.300920
9025,Astra,Omen,Phoenix,Raze,Reyna,0.297366
6903,Astra,Brimstone,Phoenix,Raze,Reyna,0.297366
1700,Astra,Omen,Raze,Reyna,Viper,0.297106
583,Brimstone,Omen,Raze,Reyna,Viper,0.236161


In [220]:
map_pick = 'Breeze'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
11168,Brimstone,Jett,Omen,Reyna,Viper,0.690743
20,Breach,Fade,Jett,Neon,Phoenix,0.676281
3453,Breach,Fade,Jett,Raze,Sova,0.668913
8611,Astra,Jett,Omen,Reyna,Viper,0.668193
2858,Breach,Fade,Jett,Neon,Sova,0.666269


In [221]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
7845,Brimstone,Chamber,Phoenix,Sage,Yoru,0.317068
2834,Brimstone,Chamber,Killjoy,Phoenix,Yoru,0.310463
6909,Brimstone,Chamber,Cypher,Phoenix,Yoru,0.307992
9839,Breach,Phoenix,Raze,Skye,Sova,0.304934
9458,Killjoy,Neon,Phoenix,Raze,Sage,0.299868


In [222]:
map_pick = 'Fracture'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
1974,Breach,KAY/O,Phoenix,Reyna,Yoru,0.673941
9697,Fade,KAY/O,Phoenix,Reyna,Yoru,0.665357
7035,Breach,KAY/O,Neon,Reyna,Yoru,0.655423
5466,KAY/O,Phoenix,Reyna,Skye,Yoru,0.653052
8625,Brimstone,Killjoy,Neon,Reyna,Sage,0.652869


In [223]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
3027,Astra,Brimstone,Chamber,Killjoy,Viper,0.298727
432,Chamber,Jett,Phoenix,Raze,Sage,0.293351
1248,Chamber,Jett,Raze,Reyna,Sage,0.289880
4243,Chamber,Jett,Neon,Reyna,Sage,0.287393
9480,Chamber,Jett,Phoenix,Sage,Yoru,0.280275


In [224]:
map_pick = 'Haven'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
8448,Breach,KAY/O,Phoenix,Sova,Yoru,0.705738
7865,Breach,KAY/O,Phoenix,Reyna,Sova,0.664798
10055,Breach,Jett,KAY/O,Phoenix,Sova,0.662187
7624,Breach,Jett,KAY/O,Neon,Sova,0.650370
1523,Astra,Neon,Omen,Phoenix,Sage,0.648911


In [225]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
4737,Chamber,Jett,Killjoy,Phoenix,Yoru,0.316446
10169,KAY/O,Phoenix,Raze,Skye,Yoru,0.312259
10784,Brimstone,Chamber,Cypher,Killjoy,Omen,0.311187
10807,Cypher,Jett,Neon,Raze,Yoru,0.310088
1864,Astra,Chamber,Phoenix,Viper,Yoru,0.305903


In [226]:
map_pick = 'Icebox'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
2641,Fade,Raze,Reyna,Skye,Yoru,0.679744
11167,Fade,Neon,Phoenix,Skye,Sova,0.674112
10061,Astra,Brimstone,Neon,Omen,Reyna,0.672672
814,Astra,Brimstone,Chamber,KAY/O,Viper,0.654045
4423,Fade,Neon,Raze,Skye,Yoru,0.653961


In [227]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
4900,Chamber,Killjoy,Omen,Phoenix,Yoru,0.306468
10325,Chamber,Cypher,Phoenix,Viper,Yoru,0.302776
2787,Jett,Neon,Reyna,Sova,Yoru,0.299651
3570,Cypher,Killjoy,Phoenix,Viper,Yoru,0.298403
11289,Brimstone,Jett,Phoenix,Viper,Yoru,0.291727


In [228]:
map_pick = 'Pearl'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
2162,Breach,Neon,Skye,Sova,Yoru,0.696288
11388,Chamber,Killjoy,Phoenix,Raze,Sage,0.670241
11267,Breach,Jett,Raze,Skye,Sova,0.668641
11352,Breach,Jett,Neon,Phoenix,Yoru,0.666997
4771,Fade,Jett,Raze,Skye,Sova,0.657280


In [229]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
9734,Chamber,Phoenix,Reyna,Sage,Yoru,0.297029
9045,Cypher,Jett,Phoenix,Reyna,Sage,0.285446
8438,Chamber,Cypher,Jett,Phoenix,Sage,0.274033
11627,Chamber,Jett,Phoenix,Reyna,Sage,0.267955
8489,Breach,Fade,Raze,Reyna,Yoru,0.252070


In [230]:
map_pick = 'Split'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
48,Astra,Chamber,Cypher,Fade,Killjoy,0.679713
11283,Astra,Brimstone,Jett,Neon,Omen,0.671943
5275,Breach,Fade,Jett,KAY/O,Raze,0.666391
1328,Chamber,Cypher,Jett,Raze,Viper,0.660335
2265,Astra,Jett,Neon,Omen,Viper,0.654609


In [231]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
901,Astra,Neon,Raze,Reyna,Viper,0.293106
1268,Astra,Jett,Neon,Reyna,Viper,0.289460
8030,Brimstone,Neon,Phoenix,Viper,Yoru,0.287935
7252,Neon,Omen,Phoenix,Viper,Yoru,0.286309
5483,Astra,Neon,Phoenix,Viper,Yoru,0.252379
